In [1]:
import numpy as np
import pandas as pd
import datetime
import math

In [10]:
dam = pd.read_csv('dam.csv')
dam

,market,date,hour,price,volume
0,1,01/01/2016,0,40.10,477.5
1,1,01/01/2016,1,40.08,486.6
2,1,01/01/2016,2,40.06,495.6
3,1,01/01/2016,3,40.08,496.6
4,1,01/01/2016,4,40.07,502.9
...,...,...,...,...,...
44587,1,29/01/2021,13,61.97,134.0
44588,1,29/01/2021,23,58.05,59.0
44589,1,30/01/2021,17,42.08,64.0
44590,1,31/01/2021,3,42.19,149.0


In [58]:
dam['date']=pd.to_datetime(dam['date'],format="%d/%m/%Y")#"%d/%m/%Y")
dam=dam.set_index(pd.DatetimeIndex(dam['date']))
dam['year']=dam.index.year
dam['month']=dam.index.month
dam['day']=dam.index.day
dam['day_of_week']=dam.index.dayofweek
dam['month_no']=dam['year']+dam['month']/100
dam.loc[dam['day_of_week']>4,'weekend']=True
dam.loc[dam['day_of_week']<=4,'weekend']=False
dam['hour_ref'] = dam.apply(lambda x: ('weekend_' + str(x['hour']) if x['weekend'] else 'weekday_' + str(x['hour'])),axis=1)
dam

,market,date,hour,price,volume,year,month,day,day_of_week,month_no,weekend,hour_ref
date,,,,,,,,,,,,
2016-01-01,1,2016-01-01,0,40.10,477.5,2016,1,1,4,2016.01,False,weekday_0
2016-01-01,1,2016-01-01,1,40.08,486.6,2016,1,1,4,2016.01,False,weekday_1
2016-01-01,1,2016-01-01,2,40.06,495.6,2016,1,1,4,2016.01,False,weekday_2
2016-01-01,1,2016-01-01,3,40.08,496.6,2016,1,1,4,2016.01,False,weekday_3
2016-01-01,1,2016-01-01,4,40.07,502.9,2016,1,1,4,2016.01,False,weekday_4
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-29,1,2021-01-29,13,61.97,134.0,2021,1,29,4,2021.01,False,weekday_13
2021-01-29,1,2021-01-29,23,58.05,59.0,2021,1,29,4,2021.01,False,weekday_23
2021-01-30,1,2021-01-30,17,42.08,64.0,2021,1,30,5,2021.01,True,weekend_17


In [68]:
monthly=dam.groupby(['month_no','year','month','hour','weekend', 'hour_ref']).mean().reset_index(['hour','weekend','month','year'])
monthly=monthly.drop(['day','day_of_week'],1)
monthly.rename(columns={'price': 'price_mean'}, inplace=True)
monthly['volume_sum']=dam[['month_no','hour_ref','volume']].groupby(['month_no','hour_ref']).sum()
monthly['hours']=dam[['month_no','hour_ref']].groupby(['month_no','hour_ref']).size()
monthly['power']=monthly['volume_sum']/monthly['hours']
monthly

year  month  hour  weekend  market  price_mean  \
month_no hour_ref                                                     
2016.01  weekday_0   2016      1     0    False     1.0   48.505238   
         weekend_0   2016      1     0     True     1.0   48.618000   
         weekday_1   2016      1     1    False     1.0   48.434286   
         weekend_1   2016      1     1     True     1.0   48.459000   
         weekday_2   2016      1     2    False     1.0   48.385714   
...                   ...    ...   ...      ...     ...         ...   
2021.01  weekend_21  2021      1    21     True     1.0   49.518000   
         weekday_22  2021      1    22    False     1.0   55.550952   
         weekend_22  2021      1    22     True     1.0   52.735000   
         weekday_23  2021      1    23    False     1.0   50.306667   
         weekend_23  2021      1    23     True     1.0   47.540000   

                         volume  volume_sum  hours       power  
month_no hour_ref                                               
2016.01  weekday_0   332.371429      6979.8     21  332.371429  
         weekend_0   372.530000      3725.3     10  372.530000  
         weekday_1   336.747619      7071.7     21  336.747619  
         weekend_1   372.900000      3729.0     10  372.900000  
         weekday_2   339.400000      7127.4     21  339.400000  
...                         ...         ...    ...         ...  
2021.01  weekend_21   84.900000       849.0     10   84.900000  
         weekday_22  109.528571      2300.1     21  109.528571  
         weekend_22  115.700000      1157.0     10  115.700000  
         weekday_23  122.947619      2581.9     21  122.947619  
         weekend_23  119.840000      1198.4     10  119.840000  

[2928 rows x 10 columns]

In [63]:
dam[['month_no','hour_ref']].groupby(['month_no','hour_ref']).size()

month_no  hour_ref  
2016.01   weekday_0     21
          weekday_1     21
          weekday_10    21
          weekday_11    21
          weekday_12    21
                        ..
2021.01   weekend_5     10
          weekend_6     10
          weekend_7     10
          weekend_8     10
          weekend_9     10
Length: 2928, dtype: int64

In [51]:
monthly.to_csv('dam_monthly.csv')

In [52]:
dam.to_csv('dam_out.csv')